In [1]:
import cv2
import pytesseract
import numpy as np
import time
import pyttsx3
import os
from gtts import gTTS
from playsound import playsound

In [2]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [3]:
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:

frames_since_last_detection = {}
previous_detected_objects=[]

In [5]:
output_layers = net.getUnconnectedOutLayersNames()
detected_objects = set()

In [6]:
engine = pyttsx3.init()

In [7]:
cap = cv2.VideoCapture(0)

In [34]:
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

True

In [8]:
def post_process(frame, outputs, conf_threshold, nms_threshold):
    height, width, _ = frame.shape
    boxes = []
    confidences = []
    class_ids = []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w // 2
                y = center_y - h // 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    return [boxes[i] for i in indices], [confidences[i] for i in indices], [class_ids[i] for i in indices]


In [ ]:
while True:
    ret, frame = cap.read()
    conf_threshold = 0.5  # Confidence threshold for filtering out weak detections
    nms_threshold = 0.4   # Non-maximum suppression threshold for eliminating overlapping boxes

    # Detect objects in the frame
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    boxes, confidences, class_ids = post_process(frame, outputs, conf_threshold, nms_threshold)

    # Update frames_since_last_detection
    for i in range(len(frames_since_last_detection)):
        object_name = list(frames_since_last_detection.keys())[i]
        if object_name in detected_objects:
            frames_since_last_detection[object_name] = 0
        else:
            frames_since_last_detection[object_name] += 1

    # Check for new detections and speak the object name
   # Check for new detections and speak the object name
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    for i in range(len(indices)):
        index = indices[i]
        class_id = class_ids[index]
        object_name = classes[class_id]
        if object_name not in detected_objects:
            detected_objects.add(object_name)
            engine.say(f"{object_name} detected")
            engine.runAndWait()
        frames_since_last_detection[object_name] = 0

    # Draw bounding boxes around the detected objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    if len(indices) == 0:
        continue
    for i in indices:
        i = indices[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)


    cv2.imshow("Object Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image to make it binary
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)

        # Make sure the contour is big enough to be a hand
        if cv2.contourArea(contour) > 10000:
            # Find the convex hull of the hand
            hull = cv2.convexHull(contour)

            # Find the defects in the convex hull
            defects = cv2.convexityDefects(contour, cv2.convexHull(contour, returnPoints=False))

            # Count the number of fingers
            num_fingers = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])

                # Calculate the length of the sides of the triangle formed by the defects
                a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

                # Calculate the angle between the sides using the law of cosines
                angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c)) * 180 / np.pi

                # If the angle is less than 90 degrees, it's a finger
                if angle < 90:
                    num_fingers += 1

            # Draw the contour and hull
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
            cv2.drawContours(frame, [hull], -1, (0, 0, 255), 2)

            # Display the number of fingers
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, str(num_fingers), (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    # Exit on ESC
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'
